<h1 style="font-size:42px; text-align:center; margin-bottom:30px;"><span style="color:SteelBlue">TM1py:</span> Data Integration</h1>
<hr>


Load Weather Data From NOAA (National Oceanic and Atmospheric Administration) API into Python.

Rearrange the data and push it to Target Cube <span style="color:SteelBlue">Weather Data</span>


## Imports

<b>requests</b> library for HTTP / REST Request against Webservices

<b>json</b> standard library for JSON parsing, manipulation


In [1]:
import requests
import json
from TM1py import TM1Service

## Constants

<b>STATION</b>  <a href="https://www.ncdc.noaa.gov/cdo-web/datasets/NORMAL_DLY/stations/GHCND:USW00014732/detail">GHCND:USW00014732</a> (40.7792°, -73.88°) 

<b>FROM, TO</b> Timeframe

<b>HEADERS</b> Token for Authentication with the API

In [2]:
STATION = 'GHCND:USW00014732'
FROM, TO = '2017-01-01', '2017-01-04'
HEADERS = {"token": 'yyqEBOAbHVbtXkfAmZuPNfnSXvdfyhgn'}

## Build URL for the Query

Build the parametrized URL and print it


In [3]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?' \
      'datasetid=GHCND&' \
      'startdate=' + FROM + '&' \
      'enddate=' + TO + '&' \
      'limit=1000&' \
      'datatypeid=TMIN&' \
      'datatypeid=TAVG&' \
      'datatypeid=TMAX&' \
      'stationid=' + STATION

print(url)

https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2017-01-01&enddate=2017-01-04&limit=1000&datatypeid=TMIN&datatypeid=TAVG&datatypeid=TMAX&stationid=GHCND:USW00014732


## Query Weather Data

Execute the URL against the NOAA API to get the results

Prettyprint first three items from result-set

In [4]:
response = requests.get(url, headers=HEADERS).json()
results = response["results"]   

print(json.dumps(results[0:3], indent=2))

[
  {
    "date": "2017-01-01T00:00:00",
    "datatype": "TAVG",
    "station": "GHCND:USW00014732",
    "attributes": "H,,S,",
    "value": 80
  },
  {
    "date": "2017-01-01T00:00:00",
    "datatype": "TMAX",
    "station": "GHCND:USW00014732",
    "attributes": ",,W,2400",
    "value": 94
  },
  {
    "date": "2017-01-01T00:00:00",
    "datatype": "TMIN",
    "station": "GHCND:USW00014732",
    "attributes": ",,W,2400",
    "value": 39
  }
]


## Rearrange Data 

Rearrange data from native JSON into a format that TM1 understands 

... Map of coordinates and value

In [5]:
cells = dict()

for record in results:
    value = record['value'] / 10
    coordinates = ("Actual", record['date'][0:10], "NYC", record['datatype'])
    cells[coordinates] = value

In [6]:
for coordinate, value in cells.items():
    print(coordinate, value)

('Actual', '2017-01-01', 'NYC', 'TAVG') 8.0
('Actual', '2017-01-01', 'NYC', 'TMAX') 9.4
('Actual', '2017-01-01', 'NYC', 'TMIN') 3.9
('Actual', '2017-01-02', 'NYC', 'TAVG') 4.4
('Actual', '2017-01-02', 'NYC', 'TMAX') 5.6
('Actual', '2017-01-02', 'NYC', 'TMIN') 3.3
('Actual', '2017-01-03', 'NYC', 'TAVG') 5.6
('Actual', '2017-01-03', 'NYC', 'TMAX') 8.3
('Actual', '2017-01-03', 'NYC', 'TMIN') 4.4
('Actual', '2017-01-04', 'NYC', 'TAVG') 8.2
('Actual', '2017-01-04', 'NYC', 'TMAX') 12.2
('Actual', '2017-01-04', 'NYC', 'TMIN') 2.2


## Send Data to TM1

Connect to TM1 and send the cellset

In [7]:
with TM1Service(address='10.77.19.60', port="12354", user="admin", password="apple", ssl=True) as tm1:
    tm1.cubes.cells.write_values("Weather Data", cells)